In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels

import os
import random

In [2]:
import sklearn
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
os.environ['PYTHONHASHSEED'] = '42'  

random.seed(42)          
np.random.seed(42)       
tensorflow.random.set_seed(42)   

In [4]:
# load the data into dataframe
df=pd.read_csv('feature_engineered_data.csv')
print(df.head())

   current_value  lights         T1       RH_1         T2       RH_2  \
0          430.0      30  20.133333  48.000000  19.566667  44.400000   
1          250.0      30  20.260000  52.726667  19.730000  45.100000   
2          100.0      10  20.426667  55.893333  19.856667  45.833333   
3          100.0      10  20.566667  53.893333  20.033333  46.756667   
4           90.0      10  20.730000  52.660000  20.166667  47.223333   

          T3       RH_3         T4       RH_4  ...  is_weekend    nsm  lag_1  \
0  19.890000  44.900000  19.000000  46.363333  ...           0  68400  576.6   
1  19.890000  45.493333  19.000000  47.223333  ...           0  69000  430.0   
2  20.033333  47.526667  19.000000  48.696667  ...           0  69600  250.0   
3  20.100000  48.466667  19.000000  48.490000  ...           0  70200  100.0   
4  20.200000  48.530000  18.926667  48.156667  ...           0  70800  100.0   

   lag_2  lag_6  lag_12  hour_sin  hour_cos  rolling_mean_12  rolling_std_12  
0  230.

In [5]:
def evaluate(model_name,target_scaler,y_test,y_pred):
   
    y_pred_actual = target_scaler.inverse_transform(y_pred)
    y_test_actual = target_scaler.inverse_transform(y_test.reshape(-1, 1))

    mae_GRU = mean_absolute_error(y_test_actual, y_pred_actual)
    r2_GRU = r2_score(y_test_actual, y_pred_actual)
    rmse_GRU = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))

    print(f"Model Name :{model_name}")
    print(f"MAE: {mae_GRU:.2f}")
    print(f"R² Score: {r2_GRU:.4f}")
    print(f"RMSE: {rmse_GRU:.4f}")

In [6]:
import pickle

# Load the selected features for 10-minute forecasting
with open("selected_features_60min.pkl", "rb") as f:
    selected_features = pickle.load(f)

print("Selected Features:", selected_features)

Selected Features: ['current_value', 'T3', 'RH_5', 'RH_8', 'T_out', 'Press_mm_hg', 'nsm', 'hour_cos', 'rolling_mean_12', 'rolling_std_12']


In [7]:
df['target_60min']= df['current_value'].shift(-6)

In [8]:
df = df.dropna().reset_index(drop=True)

In [9]:
#selected_features=selected_features.tolist()
df = df[selected_features + ['target_60min']]

In [10]:
train_size = int(0.8 * len(df))
df_train, df_test = df[:train_size], df[train_size:]

In [11]:
scaler = MinMaxScaler()
df_train_scaled = pd.DataFrame(scaler.fit_transform(df_train), columns=df.columns, index=df_train.index)
df_test_scaled = pd.DataFrame(scaler.transform(df_test), columns=df.columns, index=df_test.index)

In [12]:
def create_sequences(data, target_column, window_size):
    X, y = [], []
    for i in range(window_size, len(data)):
        X.append(data.iloc[i-window_size:i].values)
        y.append(data.iloc[i][target_column])
    return np.array(X), np.array(y)

window_size = 24  # use previous 10 time steps
X_train, y_train = create_sequences(df_train_scaled, target_column='target_60min', window_size=window_size)
X_test, y_test = create_sequences(df_test_scaled, target_column='target_60min', window_size=window_size)


In [13]:
# Invert scaling for target only
target_scaler = MinMaxScaler()
target_scaler.fit(df[['target_60min']])  # fit only on original (unscaled) appliances column

MinMaxScaler()

In [14]:
lstm = Sequential()
lstm.add(LSTM(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
lstm.add(Dense(1))  # Output layer

lstm.compile(optimizer='adam', loss='mse')
history = lstm.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, shuffle=False)


Epoch 1/20


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0296 - val_loss: 0.0122
Epoch 2/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0174 - val_loss: 0.0104
Epoch 3/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0147 - val_loss: 0.0099
Epoch 4/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0141 - val_loss: 0.0097
Epoch 5/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0139 - val_loss: 0.0096
Epoch 6/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0138 - val_loss: 0.0094
Epoch 7/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0138 - val_loss: 0.0094
Epoch 8/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0138 - val_loss: 0.0093
Epoch 9/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0138 - val_loss: 0.0092
Epoch 10/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0138 - val_loss: 0.0092
Epoch 11/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.0138 - val_loss: 0.0092
Epoch 12/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.

In [15]:
# Predict
y_pred_lstm = lstm.predict(X_test)

evaluate("LSTM",target_scaler,y_test,y_pred_lstm)


123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Model Name :LSTM
MAE: 26.97
R² Score: 0.5806
RMSE: 54.9208


In [16]:
from tensorflow.keras.layers import GRU


gru = Sequential()
gru.add(GRU(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2])))
gru.add(Dense(1))  # Output layer

gru.compile(optimizer='adam', loss='mse')
history = gru.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, shuffle=False)


Epoch 1/20


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


443/443 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0237 - val_loss: 0.0113
Epoch 2/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0147 - val_loss: 0.0103
Epoch 3/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0143 - val_loss: 0.0104
Epoch 4/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0142 - val_loss: 0.0104
Epoch 5/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0141 - val_loss: 0.0103
Epoch 6/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0140 - val_loss: 0.0101
Epoch 7/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0140 - val_loss: 0.0098
Epoch 8/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0140 - val_loss: 0.0096
Epoch 9/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0139 - val_loss: 0.0095
Epoch 10/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0139 - val_loss: 0.0094
Epoch 11/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.0139 - val_loss: 0.0093
Epoch 12/20
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.

In [17]:
# Predict
y_pred_gru = gru.predict(X_test)

evaluate("GRU",target_scaler,y_test,y_pred_gru)

123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Model Name :GRU
MAE: 26.19
R² Score: 0.5886
RMSE: 54.3983


In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tcn import TCN  # keras-tcn package

In [19]:
model_tcn = Sequential([
    TCN(input_shape=(X_train.shape[1], X_train.shape[2])),  # (timesteps, features)
    Dense(1)
])

model_tcn.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_tcn.summary()

model_tcn.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1,shuffle=False)


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\tcn\tcn.py:268: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(TCN, self).__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tcn (TCN)                       │ (None, 64)             │       138,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 138,881 (542.50 KB)

 Trainable params: 138,881 (542.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - loss: 1.6081 - mae: 0.5942 - val_loss: 0.0223 - val_mae: 0.1084
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 0.0307 - mae: 0.1264 - val_loss: 0.0160 - val_mae: 0.0822
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0245 - mae: 0.1107 - val_loss: 0.0147 - val_mae: 0.0735
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0231 - mae: 0.1075 - val_loss: 0.0153 - val_mae: 0.0722
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 0.0234 - mae: 0.1104 - val_loss: 0.0272 - val_mae: 0.1094
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.0242 - mae: 0.1121 - val_loss: 0.0337 - val_mae: 0.1341
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0250 - mae: 0.1147 - val_loss: 0.0130 - val_mae: 0.0656
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0231 - mae: 0.1090 - val_loss: 0.0121 - val_mae: 0.0612
Epoch 9/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/

In [20]:
y_pred_tcn = model_tcn.predict(X_test)

evaluate("TCN",target_scaler,y_test,y_pred_tcn)



123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Model Name :TCN
MAE: 38.42
R² Score: 0.4318
RMSE: 63.9265


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Conv1D, MaxPooling1D, Flatten, LSTM, Dense


In [22]:
model_cnn = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(1)
])

model_cnn.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_cnn.summary()


c:\Users\Rukhsana\anaconda3\envs\project\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 22, 64)         │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 704)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50)             │        35,250 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,477 (146.39 KB)

 Trainable params: 37,477 (146.39 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model_cnn.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, verbose=1,shuffle=False)


Epoch 1/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0398 - mae: 0.1295 - val_loss: 0.0153 - val_mae: 0.0666
Epoch 2/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0232 - mae: 0.0928 - val_loss: 0.0127 - val_mae: 0.0559
Epoch 3/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0192 - mae: 0.0810 - val_loss: 0.0119 - val_mae: 0.0547
Epoch 4/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0177 - mae: 0.0759 - val_loss: 0.0115 - val_mae: 0.0528
Epoch 5/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0171 - mae: 0.0743 - val_loss: 0.0118 - val_mae: 0.0504
Epoch 6/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0170 - mae: 0.0752 - val_loss: 0.0115 - val_mae: 0.0510
Epoch 7/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0168 - mae: 0.0724 - val_loss: 0.0114 - val_mae: 0.0566
Epoch 8/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0163 - mae: 0.0711 - val_loss: 0.0111 - val_mae: 0.0531
Epoch 9/10
222/222 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - lo

In [24]:
y_pred_cnn = model_cnn.predict(X_test)

evaluate("CNN",target_scaler,y_test,y_pred_cnn)


123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Model Name :CNN
MAE: 36.75
R² Score: 0.4715
RMSE: 61.6559
